In [1]:
import os

In [2]:
# Habría que hacer un mapeo de este tipo para los tags de cada categoría
# "V" debe coincidir con la posición 0 del tag que es el nombre de la categoría
# Hay que especificar la posición en el tag de cada rasgo y si hay algo para mapear, también.
# Si no hay nada para mapear con el valor que tiene en el tag en esa posición, deja el valor del tag.

tags_mapping={
    "V":{
        "subcategoria":{
            "posicion": 1
        },
        "modo": {
            "posicion": 2,
            "I": "IND",
            "S": "SUBJ",
            "M": "IMP",
            "N": "INF",
            "G": "GDIO",
            "P":"PPIO"
        },
        "tiempo": {
            "posicion": 3,
            "P":"presente",
            "I": "imperfecto",
            "F": "futuro",
            "S": "pasado",
            "C": "condicion",
            "0": "atemp"
        },
        "persona":{
            "posicion": 4
        },
        "numero":{
            "posicion": 5,
            "S":"sg",
            "P": "pl"
        }
    },
    "N":{
        "subcategoria":{ #esta subcategoría no la vamos a usar en principio porque freeling 
            # no tiene nombres propios, y puse para agregarlos aparte con otra categoría
            # PropN, pero dejo esto acá por si eventualmente queremos usarlo.
            "posicion": 1,
            "C": "COMUN", 
            "P": "PROPIO"
        },
        "genero": {
            "posicion": 2,
            "M": "masc",
            "F": "fem",
            "C": "?gen"
        },
        "numero": {
            "posicion": 3,
            "S":"sg",
            "P": "pl",
            "N": "inv"
        }
    }
}

In [3]:
class MakeRule:
    
    def __init__(self,word_dict,output="s"):
        
        self.output = output
        self.tag = word_dict.get("tag")
        self.lemma = word_dict.get("lema")
        self.shape = word_dict.get("forma")
        self.category = self.tag[0]
        self.mapping = tags_mapping.get(self.category)

        if self.mapping:
            
            self.directory = "./reglas_automaticas"
            self.file_name = f"{self.category}_rules.fcfg"
            
            for k in self.mapping.keys():
                setattr(self,k,self._get_mapped_tag(k))

            self.rule = self._make_rule()

            if self.output == "f":
                self._write_category()

            elif self.output == "s":
                print(self.rule)
                
    def _get_mapped_tag(self,rasgo):
        rasgo_mapping = self.mapping.get(rasgo)
        posicion_tag = rasgo_mapping.get("posicion")
        return rasgo_mapping.get(self.tag[posicion_tag],self.tag[posicion_tag])
                
    def _make_rule(self):
        # Habría que agregar un elif por cada categoría a la que le definamos el mapeo de keys.
        # Los atributos son las keys del diccionario. category, shape y lemma siempre se llaman
        # así porque se definen antes (es para todas las clases de palabras igual)
        # Está así cortado para que sea más legible, pero podría estar todo en una línea
        if self.category == "V":
            rule = f"{self.category}[SUBCAT={self.subcategoria},MODE={self.modo},"\
                    f"TENSE={self.tiempo},PER={self.persona},"\
                    f"NUM={self.numero},SEM=<\e.({self.lemma}(e) "\
                    f"& {self.tiempo}(e))>] -> '{self.shape}'"
        if self.category == "N":
            rule = f"{self.category}[NUM={self.numero},GEN={self.genero},"\
                    f"SEM=<\\x.({self.lemma}(x))>] -> '{self.shape}'"
        return rule
    
    def _write_category(self):
        with open(os.path.join(self.directory,self.file_name),"a+") as cat:
            cat.write(f'{self.rule}\n')
            
            

In [4]:
# Pongo esta funcionalidad porque como el guardado va agregando reglas si lo volvés
# a correr sigue sumándolas al final.

def remove_category_file(category):
    directory = "./reglas_automaticas"
    if category == "all":
        [os.remove(os.path.join(directory,f)) for f in os.listdir(directory)]
    else:
        file_name = f"{category}_rules.fcfg"
        try:
            os.remove(os.path.join(directory,file_name))
        except:
            print(f"No hay archivo {file_name} para borrar")

remove_category_file("all")
#remove_category_file("V")

In [10]:
import glob

freeling_all = list()

for freeling_file in glob.glob("./freeling/*.*.txt"):
    with open(freeling_file) as free_one:
        freeling_one = list(map(lambda a: {'forma':a[0],'lema':a[1],'tag':a[2]},
                        [f.strip("\n").split(' ') for f in free_one.readlines()]))
        freeling_all.extend(freeling_one)
        
    

# El parámetro output puede ser 's' (imprime en pantalla) 
# o 'f' (lo appendea al archivo de la clase de palabra correspondiente)

for word_index in range(len(freeling_all)):
    MakeRule(freeling_all[word_index],output="f")


{'forma': 'a', 'lema': 'a', 'tag': 'NCFS000'}
{'forma': 'ababa', 'lema': 'ababa', 'tag': 'NCFS000'}
{'forma': 'ababanes', 'lema': 'ababán', 'tag': 'NCMP000'}
{'forma': 'ababas', 'lema': 'ababa', 'tag': 'NCFP000'}
{'forma': 'ababol', 'lema': 'ababol', 'tag': 'NCMS000'}
{'forma': 'ababoles', 'lema': 'ababol', 'tag': 'NCMP000'}
{'forma': 'ababábite', 'lema': 'ababábite', 'tag': 'NCMS000'}
{'forma': 'ababábites', 'lema': 'ababábite', 'tag': 'NCMP000'}
{'forma': 'ababán', 'lema': 'ababán', 'tag': 'NCMS000'}
{'forma': 'abacera', 'lema': 'abacero', 'tag': 'NCFS000'}
{'forma': 'abaceras', 'lema': 'abacero', 'tag': 'NCFP000'}
{'forma': 'abacero', 'lema': 'abacero', 'tag': 'NCMS000'}
{'forma': 'abaceros', 'lema': 'abacero', 'tag': 'NCMP000'}
{'forma': 'abacería', 'lema': 'abacería', 'tag': 'NCFS000'}
{'forma': 'abacerías', 'lema': 'abacería', 'tag': 'NCFP000'}
{'forma': 'abacá', 'lema': 'abacá', 'tag': 'NCMS000'}
{'forma': 'abacás', 'lema': 'abacá', 'tag': 'NCMP000'}
{'forma': 'abad', 'lema': 'a

{'forma': 'agamí', 'lema': 'agamí', 'tag': 'NCMS000'}
{'forma': 'agamíes', 'lema': 'agamí', 'tag': 'NCMP000'}
{'forma': 'agamís', 'lema': 'agamí', 'tag': 'NCMP000'}
{'forma': 'agapanto', 'lema': 'agapanto', 'tag': 'NCMS000'}
{'forma': 'agapantos', 'lema': 'agapanto', 'tag': 'NCMP000'}
{'forma': 'agar', 'lema': 'agar', 'tag': 'NCMS000'}
{'forma': 'agaragar', 'lema': 'agaragar', 'tag': 'NCMS000'}
{'forma': 'agaragares', 'lema': 'agaragar', 'tag': 'NCMP000'}
{'forma': 'agarena', 'lema': 'agareno', 'tag': 'NCFS000'}
{'forma': 'agarenas', 'lema': 'agareno', 'tag': 'NCFP000'}
{'forma': 'agareno', 'lema': 'agareno', 'tag': 'NCMS000'}
{'forma': 'agarenos', 'lema': 'agareno', 'tag': 'NCMP000'}
{'forma': 'agares', 'lema': 'agar', 'tag': 'NCMP000'}
{'forma': 'agaricácea', 'lema': 'agaricácea', 'tag': 'NCFS000'}
{'forma': 'agaricáceas', 'lema': 'agaricácea', 'tag': 'NCFP000'}
{'forma': 'agarosa', 'lema': 'agarosa', 'tag': 'NCFS000'}
{'forma': 'agarosas', 'lema': 'agarosa', 'tag': 'NCFP000'}
{'form

{'forma': 'amada', 'lema': 'amado', 'tag': 'NCFS000'}
{'forma': 'amadas', 'lema': 'amado', 'tag': 'NCFP000'}
{'forma': 'amado', 'lema': 'amado', 'tag': 'NCMS000'}
{'forma': 'amados', 'lema': 'amado', 'tag': 'NCMP000'}
{'forma': 'amadrinamiento', 'lema': 'amadrinamiento', 'tag': 'NCMS000'}
{'forma': 'amadrinamientos', 'lema': 'amadrinamiento', 'tag': 'NCMP000'}
{'forma': 'amaestramiento', 'lema': 'amaestramiento', 'tag': 'NCMS000'}
{'forma': 'amaestramientos', 'lema': 'amaestramiento', 'tag': 'NCMP000'}
{'forma': 'amagadura', 'lema': 'amagadura', 'tag': 'NCFS000'}
{'forma': 'amagaduras', 'lema': 'amagadura', 'tag': 'NCFP000'}
{'forma': 'amago', 'lema': 'amago', 'tag': 'NCMS000'}
{'forma': 'amagos', 'lema': 'amago', 'tag': 'NCMP000'}
{'forma': 'amalgama', 'lema': 'amalgama', 'tag': 'NCFS000'}
{'forma': 'amalgamaciones', 'lema': 'amalgamación', 'tag': 'NCFP000'}
{'forma': 'amalgamación', 'lema': 'amalgamación', 'tag': 'NCFS000'}
{'forma': 'amalgamador', 'lema': 'amalgamador', 'tag': 'NCMS

{'forma': 'aptialismos', 'lema': 'aptialismo', 'tag': 'NCMP000'}
{'forma': 'aptitud', 'lema': 'aptitud', 'tag': 'NCFS000'}
{'forma': 'aptitudes', 'lema': 'aptitud', 'tag': 'NCFP000'}
{'forma': 'apto', 'lema': 'apto', 'tag': 'NCMS000'}
{'forma': 'aptos', 'lema': 'apto', 'tag': 'NCMP000'}
{'forma': 'apuesta', 'lema': 'apuesta', 'tag': 'NCFS000'}
{'forma': 'apuestas', 'lema': 'apuesta', 'tag': 'NCFP000'}
{'forma': 'apuntaciones', 'lema': 'apuntación', 'tag': 'NCFP000'}
{'forma': 'apuntación', 'lema': 'apuntación', 'tag': 'NCFS000'}
{'forma': 'apuntador', 'lema': 'apuntador', 'tag': 'NCMS000'}
{'forma': 'apuntadora', 'lema': 'apuntador', 'tag': 'NCFS000'}
{'forma': 'apuntadoras', 'lema': 'apuntador', 'tag': 'NCFP000'}
{'forma': 'apuntadores', 'lema': 'apuntador', 'tag': 'NCMP000'}
{'forma': 'apuntalamiento', 'lema': 'apuntalamiento', 'tag': 'NCMS000'}
{'forma': 'apuntalamientos', 'lema': 'apuntalamiento', 'tag': 'NCMP000'}
{'forma': 'apuntamiento', 'lema': 'apuntamiento', 'tag': 'NCMS000'}

{'forma': 'avenas', 'lema': 'avena', 'tag': 'NCFP000'}
{'forma': 'avenate', 'lema': 'avenate', 'tag': 'NCMS000'}
{'forma': 'avenates', 'lema': 'avenate', 'tag': 'NCMP000'}
{'forma': 'avenencia', 'lema': 'avenencia', 'tag': 'NCFS000'}
{'forma': 'avenencias', 'lema': 'avenencia', 'tag': 'NCFP000'}
{'forma': 'avenida', 'lema': 'avenida', 'tag': 'NCFS000'}
{'forma': 'avenidas', 'lema': 'avenida', 'tag': 'NCFP000'}
{'forma': 'aventador', 'lema': 'aventador', 'tag': 'NCMS000'}
{'forma': 'aventadora', 'lema': 'aventadora', 'tag': 'NCFS000'}
{'forma': 'aventadoras', 'lema': 'aventadora', 'tag': 'NCFP000'}
{'forma': 'aventadores', 'lema': 'aventador', 'tag': 'NCMP000'}
{'forma': 'aventadura', 'lema': 'aventadura', 'tag': 'NCFS000'}
{'forma': 'aventaduras', 'lema': 'aventadura', 'tag': 'NCFP000'}
{'forma': 'aventamiento', 'lema': 'aventamiento', 'tag': 'NCMS000'}
{'forma': 'aventamientos', 'lema': 'aventamiento', 'tag': 'NCMP000'}
{'forma': 'aventura', 'lema': 'aventura', 'tag': 'NCFS000'}
{'for

...

{'forma': 'cetreros', 'lema': 'cetrero', 'tag': 'NCMP000'}
{'forma': 'cetrería', 'lema': 'cetrería', 'tag': 'NCFS000'}
{'forma': 'cetrerías', 'lema': 'cetrería', 'tag': 'NCFP000'}
{'forma': 'cetro', 'lema': 'cetro', 'tag': 'NCMS000'}
{'forma': 'cetros', 'lema': 'cetro', 'tag': 'NCMP000'}
{'forma': 'cetáceo', 'lema': 'cetáceo', 'tag': 'NCMS000'}
{'forma': 'cetáceos', 'lema': 'cetáceo', 'tag': 'NCMP000'}
{'forma': 'cetárea', 'lema': 'cetárea', 'tag': 'NCFS000'}
{'forma': 'cetáreas', 'lema': 'cetárea', 'tag': 'NCFP000'}
{'forma': 'ceugma', 'lema': 'ceugma', 'tag': 'NCFS000'}
{'forma': 'ceugmas', 'lema': 'ceugma', 'tag': 'NCFP000'}
{'forma': 'ceutí', 'lema': 'ceutí', 'tag': 'NCCS000'}
{'forma': 'ceutíes', 'lema': 'ceutí', 'tag': 'NCCP000'}
{'forma': 'ceutís', 'lema': 'ceutí', 'tag': 'NCCP000'}
{'forma': 'ceñidor', 'lema': 'ceñidor', 'tag': 'NCMS000'}
{'forma': 'ceñidores', 'lema': 'ceñidor', 'tag': 'NCMP000'}
{'forma': 'ceñimiento', 'lema': 'ceñimiento', 'tag': 'NCMS000'}
{'forma': 'ceñimi

{'forma': 'citoblastos', 'lema': 'citoblasto', 'tag': 'NCMP000'}
{'forma': 'citocalasina', 'lema': 'citocalasina', 'tag': 'NCFS000'}
{'forma': 'citocalasinas', 'lema': 'citocalasina', 'tag': 'NCFP000'}
{'forma': 'citocentro', 'lema': 'citocentro', 'tag': 'NCMS000'}
{'forma': 'citocentros', 'lema': 'citocentro', 'tag': 'NCMP000'}
{'forma': 'citocinesis', 'lema': 'citocinesis', 'tag': 'NCFN000'}
{'forma': 'citocromo', 'lema': 'citocromo', 'tag': 'NCMS000'}
{'forma': 'citocromos', 'lema': 'citocromo', 'tag': 'NCMP000'}
{'forma': 'citodiéresis', 'lema': 'citodiéresis', 'tag': 'NCFN000'}
{'forma': 'citoesqueleto', 'lema': 'citoesqueleto', 'tag': 'NCMS000'}
{'forma': 'citoesqueletos', 'lema': 'citoesqueleto', 'tag': 'NCMP000'}
{'forma': 'citoféresis', 'lema': 'citoféresis', 'tag': 'NCFN000'}
{'forma': 'citogenes', 'lema': 'citogén', 'tag': 'NCMP000'}
{'forma': 'citogenética', 'lema': 'citogenética', 'tag': 'NCFS000'}
{'forma': 'citogenéticas', 'lema': 'citogenética', 'tag': 'NCFP000'}
{'form

{'forma': 'condotieros', 'lema': 'condotiero', 'tag': 'NCMP000'}
{'forma': 'condrila', 'lema': 'condrila', 'tag': 'NCFS000'}
{'forma': 'condrilas', 'lema': 'condrila', 'tag': 'NCFP000'}
{'forma': 'condrioconto', 'lema': 'condrioconto', 'tag': 'NCMS000'}
{'forma': 'condriocontos', 'lema': 'condrioconto', 'tag': 'NCMP000'}
{'forma': 'condrioma', 'lema': 'condrioma', 'tag': 'NCMS000'}
{'forma': 'condriomas', 'lema': 'condrioma', 'tag': 'NCMP000'}
{'forma': 'condriomita', 'lema': 'condriomita', 'tag': 'NCFS000'}
{'forma': 'condriomitas', 'lema': 'condriomita', 'tag': 'NCFP000'}
{'forma': 'condriomito', 'lema': 'condriomito', 'tag': 'NCMS000'}
{'forma': 'condriomitos', 'lema': 'condriomito', 'tag': 'NCMP000'}
{'forma': 'condriosoma', 'lema': 'condriosoma', 'tag': 'NCMS000'}
{'forma': 'condriosomas', 'lema': 'condriosoma', 'tag': 'NCMP000'}
{'forma': 'condritis', 'lema': 'condritis', 'tag': 'NCFN000'}
{'forma': 'condrito', 'lema': 'condrito', 'tag': 'NCMS000'}
{'forma': 'condritos', 'lema': 

{'forma': 'creditoras', 'lema': 'creditor', 'tag': 'NCFP000'}
{'forma': 'creditores', 'lema': 'creditor', 'tag': 'NCMP000'}
{'forma': 'credo', 'lema': 'credo', 'tag': 'NCMS000'}
{'forma': 'credos', 'lema': 'credo', 'tag': 'NCMP000'}
{'forma': 'credulidad', 'lema': 'credulidad', 'tag': 'NCFS000'}
{'forma': 'credulidades', 'lema': 'credulidad', 'tag': 'NCFP000'}
{'forma': 'creencia', 'lema': 'creencia', 'tag': 'NCFS000'}
{'forma': 'creencias', 'lema': 'creencia', 'tag': 'NCFP000'}
{'forma': 'crema', 'lema': 'crema', 'tag': 'NCFS000'}
{'forma': 'cremaciones', 'lema': 'cremación', 'tag': 'NCFP000'}
{'forma': 'cremación', 'lema': 'cremación', 'tag': 'NCFS000'}
{'forma': 'cremallera', 'lema': 'cremallera', 'tag': 'NCFS000'}
{'forma': 'cremalleras', 'lema': 'cremallera', 'tag': 'NCFP000'}
{'forma': 'cremas', 'lema': 'crema', 'tag': 'NCFP000'}
{'forma': 'crematorio', 'lema': 'crematorio', 'tag': 'NCMS000'}
{'forma': 'crematorios', 'lema': 'crematorio', 'tag': 'NCMP000'}
{'forma': 'crematística

KeyboardInterrupt: 

In [6]:
# Agregué esto para que arme una gramática que recoja todo y poder probar la gramática (Fernando)
# Conviene probar en otra jupyter porque si no esta se sobrecarga mucho
# Para eso armé la jupyter Test

filenames = ['GramaticaDeRasgosBase.txt', 'reglas_automaticas/V_rules.fcfg']
with open('integral_grammar.fcfg', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)